In [154]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [155]:
import boto3
import pandas as pd
import numpy as np
from IPython.display import display

"""
ec2 = boto3.client('ec2', region_name='us-west-2')
response = ec2.describe_instances()
print(response)oro
"""

def highlight_reserved(df_row):
    if df_row.name[0] in ['r4.8xlarge','m5.12xlarge']:
        return ['background-color: yellow; border: 1px  black solid !important; color: black !important']*2
    else:
        return ['border: 1px  black solid !important; color: black !important']*2
    
emr = boto3.client('emr' , region_name='us-west-2')

page_iterator = emr.get_paginator('list_clusters').paginate(
    ClusterStates=['RUNNING','WAITING']
)

clusters_active = []
for page in page_iterator:
    clusters_active.extend(page['Clusters'])

print("Total Active Clusters:", len(clusters_active))

emr_df = pd.DataFrame()
for cluster in clusters_active:
    cluster_id = cluster['Id']
    instance_groups = emr.list_instance_groups(ClusterId=cluster_id)['InstanceGroups']
    instance_groups_df = pd.DataFrame(instance_groups)
    instance_groups_df['Cluster_Name'] = cluster['Name']
    instance_groups_df['Cluster_Id'] = cluster['Id']
    emr_df = emr_df.append(instance_groups_df[['Cluster_Name','Cluster_Id','InstanceType', 'InstanceGroupType', 'Market', 'RunningInstanceCount' ]], ignore_index=True)

instance_total = emr_df['RunningInstanceCount'].sum()
print('Total Running instances count within EMR: {}'.format(instance_total))
#instance_type_cnt = instance_type_grp['RunningInstanceCount'].agg({'Count': np.sum})
#instance_type_cnt = instance_type_grp.agg({'RunningInstanceCount' : 'sum', 'Cluster_Instance' : lambda x: ', '.join(x)})
instance_grp = emr_df.groupby(['InstanceType','Market'])
#instance_grp_cnt = instance_grp.agg({'RunningInstanceCount' : 'sum'})
instance_grp_cnt = instance_grp.agg({'RunningInstanceCount' : 'sum', 'Cluster_Name' : lambda x: ', '.join(x)}).reindex(['RunningInstanceCount','Cluster_Name'], axis=1)
instance_grp_cnt = instance_grp_cnt[instance_grp_cnt['RunningInstanceCount'] != 0]
print("Count of InstanceType accross EMR")
#instance_grp_cnt = instance_grp_cnt.style.apply(highlight_reserved, axis=1)
display(instance_grp_cnt)

instance_emr_cnt = emr_df.groupby(['InstanceType','Cluster_Name','Market']).agg({'RunningInstanceCount' : 'sum'})
instance_emr_cnt = instance_emr_cnt[instance_emr_cnt['RunningInstanceCount'] != 0]
print("Distribution of InstanceType per EMR")
display(instance_emr_cnt)

emr_instance_cnt = emr_df.groupby(['Cluster_Name','InstanceType','Market']).agg({'RunningInstanceCount' : 'sum'})
emr_instance_cnt = emr_instance_cnt[emr_instance_cnt['RunningInstanceCount'] != 0]
print("Distribution of InstanceType on EMR")
display(emr_instance_cnt)

print("Distribution of Instances per Cluster")
emr_node_per_cluster = emr_df.groupby(['Cluster_Name']).agg({'RunningInstanceCount' : 'sum'})
display(emr_node_per_cluster)


Total Active Clusters: 10
Total Running instances count within EMR: 101
Count of InstanceType accross EMR


,,RunningInstanceCount,Cluster_Name
InstanceType,Market,,
i2.4xlarge,ON_DEMAND,28,"stx-usw2-ehc-prd-wafer-infosec, stx-usw2-ehc-prd-wafer-infosec"
i3.2xlarge,ON_DEMAND,3,"stx-usw2-ehc-dev-infosec-datalake-emr, stx-usw2-ehc-dev-infosec-datalake-emr"
i3.4xlarge,ON_DEMAND,1,stx-usw2-ehc-dev-infosec-datalake-emr
i3.xlarge,ON_DEMAND,3,stx-usw2-ehc-dev-infosec-datalake-emr
m4.4xlarge,ON_DEMAND,1,stx-usw2-ehc-prd-wafer-infosec
m5.12xlarge,ON_DEMAND,7,"stx-usw2-etl-ehc-spark-on-hive-prd, stx-usw2-etl-presto-prd, stx-usw2-ehc-prd-etl-T1-cluster, stx-usw2-ehc-initial-load2"
m5.24xlarge,ON_DEMAND,1,stx-usw2-etl-ehc-spark-on-hive-prd
m5.4xlarge,ON_DEMAND,1,stx-usw2-ehc-prd-etl-T1-cluster
r4.16xlarge,ON_DEMAND,6,"stx-usw2-ehc-prd-sparkdev16-h2o, stx-usw2-ehc-prd-sparkdev16-h2o"


Distribution of InstanceType per EMR


RunningInstanceCount
InstanceType Cluster_Name                          Market                         
i2.4xlarge   stx-usw2-ehc-prd-wafer-infosec        ON_DEMAND  28                  
i3.2xlarge   stx-usw2-ehc-dev-infosec-datalake-emr ON_DEMAND  3                   
i3.4xlarge   stx-usw2-ehc-dev-infosec-datalake-emr ON_DEMAND  1                   
i3.xlarge    stx-usw2-ehc-dev-infosec-datalake-emr ON_DEMAND  3                   
m4.4xlarge   stx-usw2-ehc-prd-wafer-infosec        ON_DEMAND  1                   
m5.12xlarge  stx-usw2-ehc-prd-etl-T1-cluster       ON_DEMAND  3                   
             stx-usw2-etl-ehc-spark-on-hive-prd    ON_DEMAND  3                   
             stx-usw2-etl-presto-prd               ON_DEMAND  1                   
m5.24xlarge  stx-usw2-etl-ehc-spark-on-hive-prd    ON_DEMAND  1                   
m5.4xlarge   stx-usw2-ehc-prd-etl-T1-cluster       ON_DEMAND  1                   
r4.16xlarge  stx-usw2-ehc-prd-sparkdev16-h2o       ON_DEMAND  6                   
r4.4xlarge   stx-usw2-ehc-initial-load2            ON_DEMAND  1                   
r4.8xlarge   stx-usw2-ehc-initial-load2            ON_DEMAND  2                   
             stx-usw2-ehc-prd-emrlr                ON_DEMAND  3                   
             stx-usw2-ehc-prd-sparkdev16-h2o       ON_DEMAND  1                   
             stx-usw2-etl-ehc-spark-on-hive-prd    SPOT       5                   
             stx-usw2-etl-presto-prd               ON_DEMAND  32                  
r5.12xlarge  stx-usw2-ehc-prd-etl                  ON_DEMAND  3                   
             stx-usw2-ehc-prd-etl-T2-cluster       ON_DEMAND  2                   
r5.4xlarge   stx-usw2-ehc-prd-etl-T2-cluster       ON_DEMAND  1

Distribution of InstanceType on EMR


RunningInstanceCount
Cluster_Name                          InstanceType Market                         
stx-usw2-ehc-dev-infosec-datalake-emr i3.2xlarge   ON_DEMAND  3                   
                                      i3.4xlarge   ON_DEMAND  1                   
                                      i3.xlarge    ON_DEMAND  3                   
stx-usw2-ehc-initial-load2            r4.4xlarge   ON_DEMAND  1                   
                                      r4.8xlarge   ON_DEMAND  2                   
stx-usw2-ehc-prd-emrlr                r4.8xlarge   ON_DEMAND  3                   
stx-usw2-ehc-prd-etl                  r5.12xlarge  ON_DEMAND  3                   
stx-usw2-ehc-prd-etl-T1-cluster       m5.12xlarge  ON_DEMAND  3                   
                                      m5.4xlarge   ON_DEMAND  1                   
stx-usw2-ehc-prd-etl-T2-cluster       r5.12xlarge  ON_DEMAND  2                   
                                      r5.4xlarge   ON_DEMAND  1                   
stx-usw2-ehc-prd-sparkdev16-h2o       r4.16xlarge  ON_DEMAND  6                   
                                      r4.8xlarge   ON_DEMAND  1                   
stx-usw2-ehc-prd-wafer-infosec        i2.4xlarge   ON_DEMAND  28                  
                                      m4.4xlarge   ON_DEMAND  1                   
stx-usw2-etl-ehc-spark-on-hive-prd    m5.12xlarge  ON_DEMAND  3                   
                                      m5.24xlarge  ON_DEMAND  1                   
                                      r4.8xlarge   SPOT       5                   
stx-usw2-etl-presto-prd               m5.12xlarge  ON_DEMAND  1                   
                                      r4.8xlarge   ON_DEMAND  32

Distribution of Instances per Cluster


,RunningInstanceCount
Cluster_Name,
stx-usw2-ehc-dev-infosec-datalake-emr,7
stx-usw2-ehc-initial-load2,3
stx-usw2-ehc-prd-emrlr,3
stx-usw2-ehc-prd-etl,3
stx-usw2-ehc-prd-etl-T1-cluster,4
stx-usw2-ehc-prd-etl-T2-cluster,3
stx-usw2-ehc-prd-sparkdev16-h2o,7
stx-usw2-ehc-prd-wafer-infosec,29
stx-usw2-etl-ehc-spark-on-hive-prd,9
